In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
# %rm -rf /kaggle/working/empathy_dataset_transfer
# %cd /kaggle/working
# !git clone https://bedanar:<GITHUB_TOKEN>@github.com/psytechlab/empathy_dataset_transfer.git

In [3]:
# %cd /kaggle/working/empathy_dataset_transfer
# !git checkout empathy-models

In [4]:
# import sys
# sys.path.append("/kaggle/working/empathy_dataset_transfer") 
# sys.path.append("/kaggle/working/empathy_dataset_transfer/src") 
# sys.path.append("/kaggle/working/empathy_dataset_transfer/src/contrib") 
# sys.path.append("/kaggle/working/empathy_dataset_transfer/src/contrib/empathy_models") 

In [1]:
import codecs
import numpy as np
import pandas as pd
import re
import math
import numpy as np
import random
import time
import datetime
import argparse
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split

from transformers import RobertaTokenizer
from transformers import RobertaConfig
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from src.contrib.empathy_models.models.models import BiEncoderAttentionWithRationaleClassification
from src.contrib.empathy_models.evaluation_utils import *
from transformers import AutoTokenizer

In [6]:
lr = 2e-5
lambda_EI = 1
lambda_RE = 0.5
dropout = 0.1
max_len = 128
batch_size = 4
epochs = 5
seed_val = 12
model_name = 'DeepPavlov/rubert-base-cased'

if torch.cuda.is_available():
	device = torch.device("cuda")
else:
	print('No GPU available, using the CPU instead.')
	device = torch.device("cpu")

No GPU available, using the CPU instead.


In [7]:
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [8]:
# !python /kaggle/working/empathy_dataset_transfer/src/contrib/empathy_models/process_data.py --input_path=/kaggle/input/health-empathy-dataset/emotional-reactions-reddit.csv --output_path=/kaggle/working/emotional-reactions-reddit.csv
# !python /kaggle/working/empathy_dataset_transfer/src/contrib/empathy_models/process_data.py --input_path=/kaggle/input/health-empathy-dataset/explorations-reddit.csv --output_path=/kaggle/working/explorations-reddit.csv
# !python /kaggle/working/empathy_dataset_transfer/src/contrib/empathy_models/process_data.py --input_path=/kaggle/input/health-empathy-dataset/interpretations-reddit.csv --output_path=/kaggle/working/interpretations-reddit.csv

!python src/contrib/empathy_models/process_data.py --input_path=emotional-reactions-reddit.csv --output_path=cor_emotional-reactions-reddit.csv
!python src/contrib/empathy_models/process_data.py --input_path=explorations-reddit.csv --output_path=cor_explorations-reddit.csv
!python src/contrib/empathy_models/process_data.py --input_path=interpretations-reddit.csv --output_path=cor_interpretations-reddit.csv

/Volumes/nafiska/smarty_intents/.venv/lib/python3.13/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Volumes/nafiska/smarty_intents/.venv/lib/python3.13/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncat

In [15]:
df_all = pd.read_csv('emotional-reactions-reddit.csv', delimiter=',')
# df_all = pd.read_csv('explorations-reddit.csv', delimiter=',')
# df_all = pd.read_csv('interpretations-reddit.csv', delimiter=',')

df, validation = train_test_split(df_all, train_size=0.7, random_state=42)
df_val, df_test = train_test_split(validation, test_size=0.5, random_state=42)


In [10]:
# train_path = "/kaggle/working/sample_input_ER_out.csv"
# dev_path = "/kaggle/working/sample_input_ER_out.csv"
# test_path = "/kaggle/working/sample_input_ER_out.csv"
save_model_path = '/kaggle/working/interpretations_model_output'

In [11]:
# df = pd.read_csv(train_path, delimiter=',')
df['rationale_labels'] = df['rationale_labels'].apply(lambda s: torch.tensor(np.asarray([int(i) for i in s.split(',')]), dtype=torch.long))

# df_test = pd.read_csv(test_path, delimiter=',')
df_test['rationale_labels'] = df_test['rationale_labels'].apply(lambda s: torch.tensor(np.asarray([int(i) for i in s.split(',')]), dtype=torch.long))

# df_val = pd.read_csv(dev_path, delimiter=',')
df_val['rationale_labels'] = df_val['rationale_labels'].apply(lambda s: torch.tensor(np.asarray([int(i) for i in s.split(',')]), dtype=torch.long))

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_RP = tokenizer.batch_encode_plus(df.response_post.tolist(), add_special_tokens=True, truncation=True, max_length=max_len, padding='max_length', return_attention_mask=True)
input_ids_RP = torch.tensor(tokenizer_RP['input_ids'])
attention_masks_RP = torch.tensor(tokenizer_RP['attention_mask'])

tokenizer_SP = tokenizer.batch_encode_plus(df.seeker_post.tolist(), add_special_tokens=True, truncation=True, max_length=max_len, padding='max_length', return_attention_mask=True)
input_ids_SP = torch.tensor(tokenizer_SP['input_ids'])
attention_masks_SP = torch.tensor(tokenizer_SP['attention_mask'])

In [13]:
labels = df.level.values.astype(int)
labels = torch.tensor(labels)
rationales = df.rationale_labels.values.tolist()
rationales = torch.stack(rationales, dim=0)

In [15]:
val_tokenizer_RP = tokenizer.batch_encode_plus(df_val.response_post.tolist(), add_special_tokens=True, truncation=True, max_length=max_len, padding='max_length',return_attention_mask=True)
val_input_ids_RP = torch.tensor(val_tokenizer_RP['input_ids'])
val_attention_masks_RP = torch.tensor(val_tokenizer_RP['attention_mask'])

val_tokenizer_SP = tokenizer.batch_encode_plus(df_val.seeker_post.tolist(), add_special_tokens=True, truncation=True, max_length=max_len, padding='max_length',return_attention_mask=True)
val_input_ids_SP = torch.tensor(val_tokenizer_SP['input_ids'])
val_attention_masks_SP = torch.tensor(val_tokenizer_SP['attention_mask'])

val_labels = torch.tensor(df_val.level.values.astype(int))
val_rationales = df_val.rationale_labels.values.tolist()
val_rationales = torch.stack(val_rationales, dim=0)
val_rationales_trimmed = torch.tensor(df_val.rationale_labels_trimmed.values.astype(int))

In [16]:
test_tokenizer_RP = tokenizer.batch_encode_plus(df_test.response_post.tolist(), add_special_tokens=True, truncation=True, max_length=max_len, padding='max_length', return_attention_mask=True)
test_input_ids_RP = torch.tensor(test_tokenizer_RP['input_ids'])
test_attention_masks_RP = torch.tensor(test_tokenizer_RP['attention_mask'])

test_tokenizer_SP = tokenizer.batch_encode_plus(df_test.seeker_post.tolist(), add_special_tokens=True, truncation=True, max_length=max_len, padding='max_length', return_attention_mask=True)
test_input_ids_SP = torch.tensor(test_tokenizer_SP['input_ids'])
test_attention_masks_SP = torch.tensor(test_tokenizer_SP['attention_mask'])

test_labels = torch.tensor(df_test.level.values.astype(int))
test_rationales = df_test.rationale_labels.values.tolist()
test_rationales = torch.stack(test_rationales, dim=0)
test_rationales_trimmed = torch.tensor(df_test.rationale_labels_trimmed.values.astype(int))

In [17]:
model = BiEncoderAttentionWithRationaleClassification(model_name=model_name, hidden_dropout_prob=dropout)
model = model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling 

In [18]:
params = list(model.named_parameters())
for p in model.seeker_encoder.parameters():
	p.requires_grad = False

In [19]:
optimizer = AdamW(model.parameters(), lr = lr, eps = 1e-8)

In [20]:
train_dataset = TensorDataset(input_ids_SP, attention_masks_SP, input_ids_RP, attention_masks_RP, labels, rationales)
train_size = int(len(train_dataset))
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)

val_dataset = TensorDataset(val_input_ids_SP, val_attention_masks_SP, val_input_ids_RP, val_attention_masks_RP, val_labels, val_rationales, val_rationales_trimmed)
validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

test_dataset = TensorDataset(test_input_ids_SP, test_attention_masks_SP, test_input_ids_RP, test_attention_masks_RP, test_labels, test_rationales, test_rationales_trimmed)
test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size)

In [21]:
total_steps = len(train_dataloader) * epochs
num_batch = len(train_dataloader)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
for epoch_i in range(0, epochs):
    total_train_loss = 0
    total_train_empathy_loss = 0
    total_train_rationale_loss = 0
    
    pbar = tqdm(total=num_batch, desc=f"training")
    
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids_SP = batch[0].to(device)
        b_input_mask_SP = batch[1].to(device)
        b_input_ids_RP = batch[2].to(device)
        b_input_mask_RP = batch[3].to(device)
        b_labels = batch[4].to(device)
        b_rationales = batch[5].to(device)
        
        model.zero_grad()        
        
        loss, loss_empathy, loss_rationale, logits_empathy, logits_rationale = model(input_ids_SP = b_input_ids_SP,
                                                                input_ids_RP = b_input_ids_RP, 
                                                                attention_mask_SP=b_input_mask_SP,
                                                                attention_mask_RP=b_input_mask_RP, 
                                                                empathy_labels=b_labels,
                                                                rationale_labels=b_rationales,
                                                                lambda_EI=lambda_EI,
                                                                lambda_RE=lambda_RE)
        
        
        total_train_loss += loss.item()
        total_train_empathy_loss += loss_empathy.item()
        total_train_rationale_loss += loss_rationale.item()
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix_str(
            f"total loss: {float(total_train_loss/(step+1)):.4f} epoch: {epoch_i}")
        pbar.update(1)

    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_empathy_loss = total_train_empathy_loss / len(train_dataloader)
    avg_train_rationale_loss = total_train_rationale_loss / len(train_dataloader)
    
    pbar.close()
    model.eval()
    
    total_eval_accuracy_empathy = 0
    total_eval_accuracy_rationale = 0
    
    total_pos_f1_empathy = 0.0
    total_micro_f1_empathy = 0.0
    total_macro_f1_empathy = 0.0
    
    total_pos_f1_rationale = 0.0
    total_micro_f1_rationale = 0.0
    total_macro_f1_rationale = 0.0
    
    total_iou_rationale = 0.0
    
    total_eval_loss = 0

    for batch in validation_dataloader:   
        b_input_ids_SP = batch[0].to(device)
        b_input_mask_SP = batch[1].to(device)
        b_input_ids_RP = batch[2].to(device)
        b_input_mask_RP = batch[3].to(device)
        b_labels = batch[4].to(device)
        b_rationales = batch[5].to(device)	
        b_rationales_trimmed = batch[6].to(device)	
        
        with torch.no_grad():
            loss, loss_empathy, loss_rationale, logits_empathy, logits_rationale = model(input_ids_SP = b_input_ids_SP,
                                                                input_ids_RP = b_input_ids_RP, 
                                                                attention_mask_SP=b_input_mask_SP,
                                                                attention_mask_RP=b_input_mask_RP, 
                                                                empathy_labels=b_labels,
                                                                rationale_labels=b_rationales,
                                                                lambda_EI=lambda_EI,
                                                                lambda_RE=lambda_RE)
        
        total_eval_loss += loss.item()
        
        logits_empathy = logits_empathy.detach().cpu().numpy()
        logits_rationale = logits_rationale.detach().cpu().numpy()
        
        label_empathy_ids = b_labels.to('cpu').numpy()
        label_rationale_ids = b_rationales.to('cpu').numpy()
        rationale_lens = b_rationales_trimmed.to('cpu').numpy()
        
        total_eval_accuracy_empathy += flat_accuracy(logits_empathy, label_empathy_ids, axis_=1)
        total_eval_accuracy_rationale += flat_accuracy_rationale(logits_rationale, label_rationale_ids, label_empathy_ids, rationale_lens, axis_=2)
        
        pos_f1_empathy, micro_f1_empathy, macro_f1_empathy = compute_f1(logits_empathy, label_empathy_ids, axis_=1)
        macro_f1_rationale = compute_f1_rationale(logits_rationale, label_rationale_ids, label_empathy_ids, rationale_lens, axis_=2)
        
        iou_f1_rationale = iou_f1(logits_rationale, label_rationale_ids, label_empathy_ids, rationale_lens, axis_=2)
        
        total_pos_f1_empathy += pos_f1_empathy
        total_micro_f1_empathy += micro_f1_empathy
        total_macro_f1_empathy += macro_f1_empathy
        
        total_macro_f1_rationale += macro_f1_rationale
        total_iou_rationale += iou_f1_rationale

    avg_val_accuracy_empathy = total_eval_accuracy_empathy / len(validation_dataloader)
    avg_val_accuracy_rationale = total_eval_accuracy_rationale / len(validation_dataloader)
    
    avg_val_pos_f1_empathy = total_pos_f1_empathy / len(validation_dataloader)
    avg_val_micro_f1_empathy = total_micro_f1_empathy / len(validation_dataloader)
    avg_val_macro_f1_empathy = total_macro_f1_empathy / len(validation_dataloader)
    
    avg_val_macro_f1_rationale = total_macro_f1_rationale / len(validation_dataloader)
    avg_val_iou_rationale = total_iou_rationale / len(validation_dataloader)
    
    print("  Accuracy-Empathy: {0:.4f}".format(avg_val_accuracy_empathy))
    print("  macro_f1_empathy: {0:.4f}".format(avg_val_macro_f1_empathy))
    print("  Accuracy-Rationale: {0:.4f}".format(avg_val_accuracy_rationale))
    
    print("  IOU-F1-Rationale: {0:.4f}".format(avg_val_iou_rationale))
    print("  macro_f1_rationale: {0:.4f}".format(avg_val_macro_f1_rationale))
    
    avg_val_loss = total_eval_loss / len(validation_dataloader)

In [ ]:
model.eval()

total_eval_accuracy_empathy = 0
total_eval_accuracy_rationale = 0

total_pos_f1_empathy = 0.0
total_micro_f1_empathy = 0.0
total_macro_f1_empathy = 0.0

total_pos_f1_rationale = 0.0
total_micro_f1_rationale = 0.0
total_macro_f1_rationale = 0.0
total_iou_rationale = 0.0

total_eval_loss = 0

for batch in test_dataloader:
    
    b_input_ids_SP = batch[0].to(device)
    b_input_mask_SP = batch[1].to(device)
    b_input_ids_RP = batch[2].to(device)
    b_input_mask_RP = batch[3].to(device)
    b_labels = batch[4].to(device)
    b_rationales = batch[5].to(device)
    b_rationales_trimmed = batch[6].to(device)

    with torch.no_grad():
        loss, loss_empathy, loss_rationale, logits_empathy, logits_rationale = model(input_ids_SP = b_input_ids_SP,
                                                            input_ids_RP = b_input_ids_RP, 
                                                            attention_mask_SP=b_input_mask_SP,
                                                            attention_mask_RP=b_input_mask_RP, 
                                                            empathy_labels=b_labels,
                                                            rationale_labels=b_rationales,
                                                            lambda_EI=lambda_EI,
                                                            lambda_RE=lambda_RE)

    total_eval_loss += loss.item()

    logits_empathy = logits_empathy.detach().cpu().numpy()
    logits_rationale = logits_rationale.detach().cpu().numpy()
    
    label_empathy_ids = b_labels.to('cpu').numpy()
    label_rationale_ids = b_rationales.to('cpu').numpy()
    rationale_lens = b_rationales_trimmed.to('cpu').numpy()

    total_eval_accuracy_empathy += flat_accuracy(logits_empathy, label_empathy_ids, axis_=1)
    total_eval_accuracy_rationale += flat_accuracy_rationale(logits_rationale, label_rationale_ids,  label_empathy_ids, rationale_lens, axis_=2)
    
    pos_f1_empathy, micro_f1_empathy, macro_f1_empathy = compute_f1(logits_empathy, label_empathy_ids, axis_=1)
    macro_f1_rationale = compute_f1_rationale(logits_rationale, label_rationale_ids, label_empathy_ids, rationale_lens, axis_=2)

    iou_f1_rationale = iou_f1(logits_rationale, label_rationale_ids, label_empathy_ids, rationale_lens, axis_=2)

    total_pos_f1_empathy += pos_f1_empathy
    total_micro_f1_empathy += micro_f1_empathy
    total_macro_f1_empathy += macro_f1_empathy

    total_macro_f1_rationale += macro_f1_rationale
    total_iou_rationale += iou_f1_rationale

avg_test_accuracy_empathy = total_eval_accuracy_empathy / len(test_dataloader)
avg_test_accuracy_rationale = total_eval_accuracy_rationale / len(test_dataloader)

avg_test_pos_f1_empathy = total_pos_f1_empathy / len(test_dataloader)
avg_test_micro_f1_empathy = total_micro_f1_empathy / len(test_dataloader)
avg_test_macro_f1_empathy = total_macro_f1_empathy / len(test_dataloader)

avg_test_macro_f1_rationale = total_macro_f1_rationale / len(test_dataloader)
avg_test_iou_rationale = total_iou_rationale / len(test_dataloader)

print("  Accuracy-Empathy: {0:.4f}".format(avg_test_accuracy_empathy))
print("  macro_f1_empathy: {0:.4f}".format(avg_test_macro_f1_empathy))
print("  Accuracy-Rationale: {0:.4f}".format(avg_test_accuracy_rationale))

print("  IOU-F1-Rationale: {0:.4f}".format(avg_test_iou_rationale))
print("  macro_f1_rationale: {0:.4f}".format(avg_test_macro_f1_rationale))

avg_test_loss = total_eval_loss / len(test_dataloader)

In [ ]:
torch.save(model.state_dict(), save_model_path)

# inference

In [ ]:
from src.contrib.empathy_models.empathy_classifier import EmpathyClassifier
import csv

In [ ]:
input_path = '/kaggle/working/Empathy-Mental-Health/dataset/sample_test_input.csv'
output_path = '/kaggle/working/sample_test_input_out.csv'
ER_model_path = "/kaggle/working/model_output"
IP_model_path = "/kaggle/working/interpretations_model_output"
EX_model_path = "/kaggle/working/explorations_model_output"

In [ ]:
# input_df = pd.read_csv(input_path, header=0)
input_df = df_test

In [ ]:
ids = input_df.id.astype(str).tolist()
seeker_posts = input_df.seeker_post.astype(str).tolist()
response_posts = input_df.response_post.astype(str).tolist()

In [ ]:
empathy_classifier = EmpathyClassifier(device,
						ER_model_path = ER_model_path, 
						IP_model_path = IP_model_path,
						EX_model_path = EX_model_path,)

In [ ]:
output_file = codecs.open(output_path, 'w', 'utf-8')
csv_writer = csv.writer(output_file, delimiter=',', quotechar='"')
csv_writer.writerow(['id','seeker_post','response_post','ER_label','IP_label','EX_label', 'ER_rationale', 'IP_rationale', 'EX_rationale'])

In [ ]:
for i in range(len(seeker_posts)):
	(logits_empathy_ER, predictions_ER, logits_empathy_IP, predictions_IP, logits_empathy_EX, predictions_EX, logits_rationale_ER, predictions_rationale_ER, logits_rationale_IP, predictions_rationale_IP, logits_rationale_EX,predictions_rationale_EX) = empathy_classifier.predict_empathy([seeker_posts[i]], [response_posts[i]])

	csv_writer.writerow([ids[i], seeker_posts[i], response_posts[i], predictions_ER[0], predictions_IP[0], predictions_EX[0], predictions_rationale_ER[0].tolist(), predictions_rationale_IP[0].tolist(), predictions_rationale_EX[0].tolist()])

In [ ]:
output_file.close()

In [ ]:
t = pd.read_csv(output_file)

In [ ]:
t.head()